In [2]:
import av
import cv2
import os
import torch
import pyiqa
import numpy as np
import json
from collections import deque
from tqdm import tqdm
from scenedetect import open_video, SceneManager
from scenedetect.detectors import AdaptiveDetector
from colorama import Fore, Style, init

# Initialize colorama for colored output
init(autoreset=True)

def print_video_info(file_path, container, video_stream, frame_rate, total_frames):
    """Display video file information and metadata"""
    print(f"{Fore.CYAN}{'='*60}")
    print(f"{Fore.CYAN}VIDEO INFORMATION")
    print(f"{Fore.CYAN}{'='*60}")
    print(f"{Fore.WHITE}File: {Fore.YELLOW}{os.path.basename(file_path)}")
    print(f"{Fore.WHITE}Format: {Fore.GREEN}{container.format.name}")
    print(f"{Fore.WHITE}Duration: {Fore.GREEN}{container.duration / av.time_base:.2f} seconds")
    print(f"{Fore.WHITE}Codec: {Fore.GREEN}{video_stream.codec.name}")
    print(f"{Fore.WHITE}Resolution: {Fore.GREEN}{video_stream.width}x{video_stream.height}")
    print(f"{Fore.WHITE}Frame Rate: {Fore.GREEN}{frame_rate} fps")
    print(f"{Fore.WHITE}Pixel Format: {Fore.GREEN}{video_stream.pix_fmt}")
    print(f"{Fore.WHITE}Bitrate: {Fore.GREEN}{container.bit_rate} bps")
    print(f"{Fore.WHITE}Total Frames: {Fore.GREEN}{total_frames}")
    print(f"{Fore.CYAN}{'='*60}{Style.RESET_ALL}")

def convert_frame_to_720p(frame):
    """Convert frame to 720p resolution at 8-bit depth"""
    frame_720p = frame.reformat(width=1280, height=720, format='yuv420p')
    return frame_720p.to_ndarray()

def read_mxf_video(file_path):
    """Read MXF video file and convert frames to 720p"""
    print(f"{Fore.MAGENTA}🎬 Opening MXF video file...")
    
    # Open video containers
    container = av.open(file_path)
    video_stream = container.streams.video[0]
    
    # Get accurate frame info using OpenCV
    cap = cv2.VideoCapture(file_path)
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    
    print_video_info(file_path, container, video_stream, frame_rate, total_frames)
    
    # Prepare video metadata
    video_info = {
        'file_path': file_path,
        'format': container.format.name,
        'codec': video_stream.codec.name,
        'original_resolution': f"{video_stream.width}x{video_stream.height}",
        'frame_rate': frame_rate,
        'pixel_format': video_stream.pix_fmt,
        'bitrate': container.bit_rate,
        'total_frames': total_frames
    }
    
    print(f"{Fore.BLUE}🔄 Converting frames to 720p...")
    frame_count = 0
    frames_720p = []
    
    # Process each frame with progress tracking
    with tqdm(total=total_frames, desc="Processing frames", unit="frame", colour="blue") as pbar:
        for frame in container.decode(video_stream):
            frame_count += 1
            converted_frame = convert_frame_to_720p(frame)
            frames_720p.append(converted_frame)
            pbar.update(1)
    
    container.close()
    print(f"{Fore.GREEN}✅ Successfully processed {frame_count} frames and converted to 720p")
    return frames_720p, video_info

def find_and_split_scenes(video_path, frame_rate):
    """Detect scenes in video using adaptive threshold"""
    print(f"{Fore.MAGENTA}🎯 Starting scene detection...")
    
    # Initialize scene detection
    video = open_video(video_path)
    scene_manager = SceneManager()
    scene_manager.add_detector(AdaptiveDetector(adaptive_threshold=3.0))
    
    print(f"{Fore.BLUE}🔍 Analyzing video for scene changes...")
    scene_manager.detect_scenes(video=video, show_progress=True)
    
    # Process detected scenes
    scene_list = scene_manager.get_scene_list()
    print(f"{Fore.GREEN}📊 Found {Fore.YELLOW}{len(scene_list)}{Fore.GREEN} scenes")
    
    scenes_info = []
    if scene_list:
        for i, (start_time, end_time) in enumerate(scene_list):
            start_frame = int(start_time.get_seconds() * frame_rate)
            end_frame = int(end_time.get_seconds() * frame_rate)
            scenes_info.append({
                'scene_id': i + 1,
                'start_time_seconds': round(start_time.get_seconds(), 2),
                'end_time_seconds': round(end_time.get_seconds(), 2),
                'start_frame': start_frame,
                'end_frame': end_frame,
                'frame_count': end_frame - start_frame + 1
            })
            print(f"{Fore.CYAN}  Scene {i+1}: Frames {start_frame}-{end_frame} ({end_frame-start_frame+1} frames)")
    else:
        print(f"{Fore.RED}⚠️  No scenes detected")
    
    return scenes_info

def find_sequence_per_scene(frames_list, scenes_info, base_video_name, musiq_metric, niqe_metric, device, sequence_length=15, musiq_threshold=35.0, niqe_threshold=6.0):
    """Find high-quality frame sequences in each detected scene"""
    scene_results = []
    
    print(f"{Fore.MAGENTA}🎨 Starting quality analysis per scene...")
    print(f"{Fore.CYAN}   Thresholds: MUSIQ > {musiq_threshold}, NIQE < {niqe_threshold}")
    print(f"{Fore.CYAN}   Target sequence length: {sequence_length} frames")
    
    # Process each scene individually
    for scene in tqdm(scenes_info, desc="Analyzing scenes", unit="scene", colour="magenta"):
        scene_id = scene['scene_id']
        start_frame = scene['start_frame']
        end_frame = scene['end_frame']
        
        print(f"{Fore.BLUE}🔍 Processing Scene {scene_id} (frames {start_frame}-{end_frame})")
        
        frame_buffer = deque(maxlen=sequence_length)
        selected_frames = []
        
        # Analyze frames within current scene
        for frame_idx in range(start_frame - 1, min(end_frame, len(frames_list))):
            frame_array = frames_list[frame_idx]
            frame_number = frame_idx + 1
            
            # Skip low-variance frames (likely blank/black)
            if np.std(frame_array) < 10.0:
                frame_buffer.clear()
                continue
            
            # Convert frame for quality analysis
            frame_bgr = cv2.cvtColor(frame_array, cv2.COLOR_YUV2BGR_I420)
            frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
            frame_tensor = torch.tensor(frame_rgb).permute(2, 0, 1).unsqueeze(0) / 255.0
            frame_tensor = frame_tensor.to(device)
            
            # Calculate quality metrics with error handling for NIQE
            with torch.no_grad():
                musiq_score = musiq_metric(frame_tensor).item()
                
                # Try NIQE calculation, skip frame if it fails
                try:
                    niqe_score = niqe_metric(frame_tensor).item()
                except:
                    # Skip this frame if NIQE calculation fails
                    frame_buffer.clear()
                    continue
            
            # Check if frame meets quality thresholds
            is_good_quality = (musiq_score >= musiq_threshold) and (niqe_score <= niqe_threshold)
            
            if is_good_quality:
                frame_buffer.append(frame_number)
            else:
                frame_buffer.clear()
            
            # Check if we found a complete sequence
            if len(frame_buffer) == sequence_length:
                selected_frames = list(frame_buffer)
                break
        
        # Store results for current scene
        success = len(selected_frames) == sequence_length
        scene_result = {
            'scene_id': scene_id,
            'sequence_found': success,
            'selected_frames': selected_frames,
            'total_frames_selected': len(selected_frames)
        }
        
        if success:
            print(f"{Fore.GREEN}✅ Scene {scene_id}: Found {sequence_length} consecutive high-quality frames")
        else:
            print(f"{Fore.YELLOW}⚠️  Scene {scene_id}: Only found {len(selected_frames)} quality frames")
        
        scene_results.append(scene_result)
    
    return scene_results

def save_analysis_json(video_info, scenes_info, scene_results, output_path):
    """Save complete analysis results to JSON file"""
    print(f"{Fore.MAGENTA}💾 Preparing analysis data for export...")
    
    # Add frame selection info to each scene
    for scene in scenes_info:
        scene_id = scene['scene_id']
        scene_result = next((sr for sr in scene_results if sr['scene_id'] == scene_id), None)
        
        if scene_result:
            scene['frames_selected'] = scene_result['sequence_found']
            if scene_result['sequence_found']:
                scene['selected_frame_range'] = {
                    'start_frame': scene_result['selected_frames'][0],
                    'end_frame': scene_result['selected_frames'][-1],
                    'total_selected': len(scene_result['selected_frames'])
                }
            else:
                scene['selected_frame_range'] = None
        else:
            scene['frames_selected'] = False
            scene['selected_frame_range'] = None
    
    # Calculate summary statistics
    total_sequences_found = sum(1 for sr in scene_results if sr['sequence_found'])
    total_frames_selected = sum(sr['total_frames_selected'] for sr in scene_results)
    
    # Prepare final analysis data
    analysis_data = {
        'video_information': video_info,
        'scene_detection': {
            'total_scenes_detected': len(scenes_info),
            'scenes': scenes_info
        },
        'quality_analysis': {
            'total_scenes_with_sequences': total_sequences_found,
            'total_frames_selected': total_frames_selected,
            'scene_results': scene_results
        }
    }
    
    # Save to JSON file
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w') as f:
        json.dump(analysis_data, f, indent=2)
    
    print(f"{Fore.GREEN}📄 Analysis data saved to: {Fore.YELLOW}{output_path}")
    return analysis_data

def process_mxf_complete_pipeline(mxf_file_path, output_folder):
    """Complete MXF analysis pipeline: read video, detect scenes, analyze frame quality"""
    
    print(f"{Fore.MAGENTA}{'='*80}")
    print(f"{Fore.MAGENTA}🚀 STARTING MXF VIDEO ANALYSIS PIPELINE")
    print(f"{Fore.MAGENTA}{'='*80}{Style.RESET_ALL}")
    
    # Step 1: Read and convert MXF video
    print(f"\n{Fore.CYAN}📖 STEP 1: Reading MXF file and converting frames")
    frames_list, video_info = read_mxf_video(mxf_file_path)
    
    # Step 2: Detect video scenes
    print(f"\n{Fore.CYAN}🎬 STEP 2: Scene detection")
    scenes_info = find_and_split_scenes(mxf_file_path, video_info['frame_rate'])
    
    # Step 3: Initialize quality analysis models
    print(f"\n{Fore.CYAN}🤖 STEP 3: Quality-based frame analysis per scene")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"{Fore.BLUE}🔧 Initializing IQA models on device: {Fore.GREEN}{device}")
    
    musiq_metric = pyiqa.create_metric('musiq', device=device)
    niqe_metric = pyiqa.create_metric('niqe', device=device)
    print(f"{Fore.GREEN}✅ MUSIQ and NIQE models loaded successfully")
    
    # Step 4: Analyze frame quality per scene
    base_video_name = os.path.basename(mxf_file_path).split('.')[0]
    scene_results = find_sequence_per_scene(
        frames_list, 
        scenes_info,
        base_video_name,
        musiq_metric, 
        niqe_metric, 
        device
    )
    
    # Step 5: Save results
    print(f"\n{Fore.CYAN}💾 STEP 4: Saving analysis results")
    json_output_path = os.path.join(output_folder, f"{base_video_name}_analysis.json")
    analysis_data = save_analysis_json(video_info, scenes_info, scene_results, json_output_path)
    
    # Final summary
    sequences_found = sum(1 for sr in scene_results if sr['sequence_found'])
    total_scenes = len(scenes_info)
    
    print(f"\n{Fore.GREEN}{'='*80}")
    print(f"{Fore.GREEN}🎉 PIPELINE COMPLETED SUCCESSFULLY!")
    print(f"{Fore.GREEN}📊 Results: Found quality sequences in {Fore.YELLOW}{sequences_found}/{total_scenes}{Fore.GREEN} scenes")
    print(f"{Fore.GREEN}{'='*80}{Style.RESET_ALL}")
    
    return analysis_data

if __name__ == "__main__":
    mxf_file = r"movie\Beast.mkv"
    output_folder = r"analysis_results"
    
    analysis_data = process_mxf_complete_pipeline(mxf_file, output_folder)

🚀 STARTING MXF VIDEO ANALYSIS PIPELINE

📖 STEP 1: Reading MXF file and converting frames
🎬 Opening MXF video file...
VIDEO INFORMATION
File: Beast.mkv
Format: matroska,webm
Duration: 218.89 seconds
Codec: h264
Resolution: 1920x1080
Frame Rate: 25.0 fps
Pixel Format: yuv420p
Bitrate: 1166371 bps
Total Frames: 5472
🔄 Converting frames to 720p...


Processing frames: 100%|          | 0/5472 [00:00<?, ?frame/s]          | 38/5472 [00:00<00:14, 378.51frame/s]▏         | 83/5472 [00:00<00:12, 419.19frame/s]▏         | 129/5472 [00:00<00:12, 437.67frame/s]▎         | 173/5472 [00:00<00:12, 423.93frame/s]▍         | 220/5472 [00:00<00:11, 437.80frame/s]▍         | 266/5472 [00:00<00:11, 445.17frame/s]▌         | 311/5472 [00:00<00:11, 439.46frame/s]▋         | 355/5472 [00:00<00:11, 434.66frame/s]▋         | 399/5472 [00:00<00:11, 430.37frame/s]▊         | 446/5472 [00:01<00:11, 441.63frame/s]▉         | 491/5472 [00:01<00:11, 443.44frame/s]▉         | 539/5472 [00:01<00:10, 454.31frame/s]█         | 585/5472 [00:01<00:11, 437.89frame/s]█▏        | 629/5472 [00:01<00:11, 436.14frame/s]█▏        | 673/5472 [00:01<00:11, 433.25frame/s]█▎        | 717/5472 [00:01<00:11, 429.55frame/s]█▍        | 764/5472 [00:01<00:10, 439.43frame/s]█▍        | 808/5472 [00:01<00:10, 431.35frame/s]█▌        | 852/5472 [00:01<00:11, 419.77frame/s]█▋       

✅ Successfully processed 5471 frames and converted to 720p

🎬 STEP 2: Scene detection
🎯 Starting scene detection...
🔍 Analyzing video for scene changes...


  Detected: 64 | Progress: 100%|█████████▉| 5471/5472 [00:14<00:00, 370.39frames/s]


📊 Found 65 scenes
  Scene 1: Frames 0-135 (136 frames)
  Scene 2: Frames 135-343 (209 frames)
  Scene 3: Frames 343-386 (44 frames)
  Scene 4: Frames 386-541 (156 frames)
  Scene 5: Frames 541-679 (139 frames)
  Scene 6: Frames 679-927 (249 frames)
  Scene 7: Frames 927-1100 (174 frames)
  Scene 8: Frames 1100-1194 (95 frames)
  Scene 9: Frames 1194-1322 (129 frames)
  Scene 10: Frames 1322-1386 (65 frames)
  Scene 11: Frames 1386-1505 (120 frames)
  Scene 12: Frames 1505-1547 (43 frames)
  Scene 13: Frames 1547-1636 (90 frames)
  Scene 14: Frames 1636-1850 (215 frames)
  Scene 15: Frames 1850-1910 (61 frames)
  Scene 16: Frames 1910-1942 (33 frames)
  Scene 17: Frames 1942-1979 (38 frames)
  Scene 18: Frames 1979-2062 (84 frames)
  Scene 19: Frames 2062-2123 (62 frames)
  Scene 20: Frames 2123-2179 (57 frames)
  Scene 21: Frames 2179-2293 (115 frames)
  Scene 22: Frames 2293-2492 (200 frames)
  Scene 23: Frames 2492-2694 (203 frames)
  Scene 24: Frames 2694-2749 (56 frames)
  Scene 25

...

🔍 Processing Scene 65 (frames 5347-5471)


c:\Users\Welcome\anaconda3\envs\qube_cinemas\lib\site-packages\pyiqa\matlab_utils\functions.py:173: UserWarning: cov(): degrees of freedom is <= 0. Correction should be strictly less than the number of observations. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\Correlation.cpp:116.)
  return torch.cov(tensor, correction=correction)
c:\Users\Welcome\anaconda3\envs\qube_cinemas\lib\site-packages\pyiqa\archs\niqe_arch.py:161: UserWarning: torch.linalg.svd: During SVD computation with the selected cusolver driver, batches 0 failed to converge. A more accurate method will be used to compute the SVD as a fallback. Check doc at https://pytorch.org/docs/stable/generated/torch.linalg.svd.html (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\cuda\linalg\BatchLinearAlgebraLib.cpp:696.)
  invcov_param = torch.linalg.pinv((cov_pris_param + cov_distparam) / 2)


⚠️  Scene 65: Only found 0 quality frames


Analyzing scenes: 100%|██████████| 65/65 [01:54<00:00,  1.08s/scene]██████████| 65/65 [01:54<00:00,  1.76s/scene]



💾 STEP 4: Saving analysis results
💾 Preparing analysis data for export...
📄 Analysis data saved to: analysis_results\Beast_analysis.json

🎉 PIPELINE COMPLETED SUCCESSFULLY!
📊 Results: Found quality sequences in 33/65 scenes
